In [37]:
import numpy as np 
import pandas as pd
import matplotlib as plt
import math
import re
from collections import Counter
from sklearn.decomposition import TruncatedSVD

In [38]:
credits=pd.read_csv(r'F:\NLP project\tmdb_5000_credits.csv\tmdb_5000_credits.csv',encoding_errors = 'ignore')
movies=pd.read_csv(r'F:\NLP project\tmdb_5000_movies.csv\tmdb_5000_movies.csv',encoding_errors = 'ignore')
credits

movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4798      9367                               El Mariachi   
4799     72766                                 Newlyweds   
4800    231617                 Signed, Sealed, Delivered   
4801    126186                          Shanghai Calling   
4802     25975                         My Date with Drew   

                                                   cast  \
0     [{"cast_id": 242, "character": "Jake Sully", "...   
1     [{"cast_id": 4, "character": "Captain Jack Spa...   
2     [{"cast_id": 1, "character": "James Bond", "cr...   
3     [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4     [{"cast_id": 5, "character": "John Carter", "c...   
...                                                 ...   
4798  [{"cast_id": 1, "character": "El Mariachi", "c...   
4799  [{"cast_id": 1, "character": "Buzzy", "credit_...   
4800  [{"cast_id": 8, "character": "Oliver O\u2019To...   
4801  [{"cast_id": 3, "character": "Sam", "credit_id...   
4802  [{"cast_id": 3, "character": "Herself", "credi...   

                                                   crew  
0     [{"credit_id": "52fe48009251416c750aca23", "de...  
1     [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2     [{"credit_id": "54805967c3a36829b5002c41", "de...  
3     [{"credit_id": "52fe4781c3a36847f81398c3", "de...  
4     [{"credit_id": "52fe479ac3a36847f813eaa3", "de...  
...                                                 ...  
4798  [{"credit_id": "52fe44eec3a36847f80b280b", "de...  
4799  [{"credit_id": "52fe487dc3a368484e0fb013", "de...  
4800  [{"credit_id": "52fe4df3c3a36847f8275ecf", "de...  
4801  [{"credit_id": "52fe4ad9c3a368484e16a36b", "de...  
4802  [{"credit_id": "58ce021b9251415a390165d9", "de...  

[4803 rows x 4 columns]

In [39]:
credits_column_renamed=credits.rename(columns={"movie_id":"id"})
movies_df_merged=movies.merge(credits_column_renamed,on='id')
movies_df_merged.head()

budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3  [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4  [{"id": 818, "name": "based on novel"}, {"id":...                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   
2  A cryptic message from Bond’s past sends him o...  107.376788   
3  Following the death of District Attorney Harve...   112.31295   
4  John Carter is a war-weary, former military ca...   43.926995   

                                production_companies  ... vote_count label  \
0  [{"name": "Ingenious Film Partners", "id": 289...  ...      11800     0   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...  ...       4500     0   
2  [{"name": "Columbia Pictures", "id": 5}, {"nam...  ...       4466     0   
3  [{"name": "Legendary Pictures", "id": 923}, {"...  ...       9106     0   
4        [{"name": "Walt Disney Pictures", "id": 2}]  ...       2124     0   

  Unnamed: 21 Unnamed: 22 Unnamed: 23 Unnamed: 24 Unnamed: 25  \
0         NaN         NaN         NaN         NaN         NaN   
1         NaN         NaN         NaN         NaN         NaN   
2         NaN         NaN         NaN         NaN         NaN   
3         NaN         NaN         NaN         NaN         NaN   
4         NaN         NaN         NaN         NaN         NaN   

                                    title_y  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   
2  [{"cast_id": 1, "character": "James Bond", "cr...   
3  [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4  [{"cast_id": 5, "character": "John Carter", "c...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2  [{"credit_id": "54805967c3a36829b5002c41", "de...  
3  [{"credit_id": "52fe4781c3a36847f81398c3", "de...  
4  [{"credit_id": "52fe479ac3a36847f813eaa3", "de...  

[5 rows x 29 columns]

In [40]:
movies_clean=movies_df_merged[['id','original_title','overview']]
movies_clean

id                            original_title  \
0      19995                                    Avatar   
1        285  Pirates of the Caribbean: At World's End   
2     206647                                   Spectre   
3      49026                     The Dark Knight Rises   
4      49529                               John Carter   
...      ...                                       ...   
4797    9367                               El Mariachi   
4798   72766                                 Newlyweds   
4799  231617                 Signed, Sealed, Delivered   
4800  126186                          Shanghai Calling   
4801   25975                         My Date with Drew   

                                               overview  
0     In the 22nd century, a paraplegic Marine is di...  
1     Captain Barbossa, long believed to be dead, ha...  
2     A cryptic message from Bond’s past sends him o...  
3     Following the death of District Attorney Harve...  
4     John Carter is a war-weary, former military ca...  
...                                                 ...  
4797  El Mariachi just wants to play his guitar and ...  
4798  A newlywed couple's honeymoon is upended by th...  
4799  "Signed, Sealed, Delivered" introduces a dedic...  
4800  When ambitious New York attorney Sam is sent t...  
4801  Ever since the second grade when he first saw ...  

[4802 rows x 3 columns]

In [41]:
col=movies_clean.columns
res=[]
for c in col:
    d=movies_clean[col].values.tolist()
    res.append(d)
res

[[[19995,
   'Avatar',
   'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization.'],
  [285,
   "Pirates of the Caribbean: At World's End",
   'Captain Barbossa, long believed to be dead, has come back to life and is headed to the edge of the Earth with Will Turner and Elizabeth Swann. But nothing is quite as it seems.'],
  [206647,
   'Spectre',
   'A cryptic message from Bond’s past sends him on a trail to uncover a sinister organization. While M battles political forces to keep the secret service alive, Bond peels back the layers of deceit to reveal the terrible truth behind SPECTRE.'],
  [49026,
   'The Dark Knight Rises',
   "Following the death of District Attorney Harvey Dent, Batman assumes responsibility for Dent's crimes to protect the late attorney's reputation and is subsequently hunted by the Gotham City Police Department. Eight years later, Batman encou

In [42]:
def tf(word, count):
    return count[word] / sum(count.values())


def n_containing(word, count_list):
    return sum(1 for count in count_list if word in count)
 
 
def idf(word, count_list):
    return math.log(len(count_list) / (1 + n_containing(word, count_list)))


def tfidf(word, count, count_list):
    return tf(word, count) * idf(word, count_list)


def as_num(x):
    y='{:.5f}'.format(x)
    return(y)


def cosine_similarity(x,y):
    num = x.dot(y.T)
    denom = np.linalg.norm(x) * np.linalg.norm(y)
    cos = num/ denom
    return cos


def euclideandistance(x,y):
    return np.sqrt(np.sum(np.square(x-y)))


In [43]:
stopwords = []
movie_tfidfvector=[]
with open("stopword.txt", "r") as f:
    while True:
        line = f.readline()
        if not line:
            break
        stopwords.append(line.strip("\n").strip())
countlist = []
for i in range(0,1470):
    with open(r"F:\NLP project\movie/"+str(i)+".txt", "r",encoding='utf-8') as f:    
        data = f.read()   
        
    text = re.sub(r"'s", " 's", data)
    text = re.sub(r"'ve", " 've", data)
    text = re.sub(r"n't", " n't", data)
    text = re.sub(r"'re", " 're", data)
    text = re.sub(r"'d", " 'd", data)
    text = re.sub(r"'ll", " 'll", data)
    text = re.sub(r"[^A-Za-z]", " ", data)
    do_lower_case=True
    if do_lower_case:
        text=text.strip().lower()
    tokens = text.split()
    text_words = [token for token in tokens if token not in stopwords]
    count = Counter(text_words)
    countlist.append(count)
    docvector=np.array([tfidf(word, count, countlist) for word in count])
    docvector1=np.reshape(docvector,(1,len(docvector)))
    docvector2 = abs(np.sort(-docvector1))
    movie_tfidfvector.append(docvector2[0][:5])

In [44]:
indices=pd.Series(movies_clean.index,index=movies_clean.original_title)

In [45]:
vector=movie_tfidfvector[4]
vector_new=vector.reshape(1,5)
movie_tfidfvector_linshi=np.array(movie_tfidfvector)
movie_tfidfvector_new=movie_tfidfvector_linshi.T
movie_tfidfvector_new.shape

(5, 1470)

In [46]:
svd = TruncatedSVD(n_components = 3)
svdMatrix = svd.fit_transform(movie_tfidfvector_linshi)
svdMatrix[3]

array([ 0.07216203, -0.0151446 , -0.0027677 ])

In [47]:
def recommend(movie):
    movie_tfidfvector_linshi=np.array(movie_tfidfvector)
    svd = TruncatedSVD(n_components = 3)
    svdMatrix = svd.fit_transform(movie_tfidfvector_linshi)
    targetmov=svdMatrix[indices[movie]]
    score=[]
    for i,j in enumerate(svdMatrix):
        score.append([i,euclideandistance(j,targetmov)])
    Relatedmovie=sorted(score,key = lambda x: x[1],reverse = False)   
    res=Relatedmovie[1:11]
    movies_indices = [i[0] for i in res]
    print("Movie related to %s"%(movie))
    return movies_clean.original_title.iloc[movies_indices]
  

In [48]:
recommend('Avatar')

Movie related to Avatar


7                Avengers: Age of Ultron
3                  The Dark Knight Rises
4                            John Carter
6                                Tangled
9     Batman v Superman: Dawn of Justice
2                                Spectre
27                            Battleship
5                           Spider-Man 3
29                               Skyfall
16                          The Avengers
Name: original_title, dtype: object